# PROYECTO FINAL APRENDIZAJE AUTOMÁTICO

---

## PREPROCESAMIENTO DEL DATASET

**Autor**: Andrés Gil Vicente

**Fecha de entrega**: 04/05/2025

---

### Objetivos:

Este fichero tiene como objetivo realizar el preprocesamiento del conjunto de datos original con el fin de preparar la información para su posterior análisis y modelado. Las tareas llevadas a cabo en este módulo son fundamentales para garantizar la calidad, consistencia y utilidad de los datos, minimizando errores y maximizando la eficacia de los modelos predictivos.

A lo largo de este proceso, se llevarán a cabo las siguientes tareas:

- Carga del conjunto de datos desde su fuente original.

- Breve exploración de la estructura, tipos de variables y valores ausentes (basándonos en las conclusiones del exploracion.ipynb)

- Limpieza de datos, incluyendo:

    - Detección y tratamiento de valores nulos.
    - Tratado de valores físicamente imposibles
    - Imputación de datos inconsistentes o atípicos.

- Codificación de variables categóricas mediante técnicas de dummy encoding.

- Split del dataset en conjunto de train y conjunto de validación.

- Estandarización de variables.

- Almacenamiento de los datos en CSV.

---

### Resultados:

El resultado final de este proceso serán 2 conjuntos de datos limpios, estructurados y listos para ser utilizado para la posterior construcción de nuestros modelos predictivos. Tendremos por un lado el conjunto de train y por otro lado el conjunto de validación. Además, generaremos también un archivo csv donde almacenaremos los datos de estandarización que hemos empleado durante el proceso, es decir, la media y la desviación típica de cada columna del conjunto de train.


### Importamos las librerías necesarias:

In [1]:
from funciones import*

### Cargamos los ficheros de datos:

In [2]:
# Leemos los ficheros y los convertimos a dataframe. Las variables se pueden cambiar si las rutas son distintas
carpeta_datos = "data"
df_train = pd.read_csv(f"{carpeta_datos}/rendimiento_estudiantes_train.csv", delimiter=",")

# Definimos cual es la variable objetivo
variable_objetivo = "T3"


In [3]:
# Mostramos las primeras filas del dataset para ver qué aspecto tiene
df_train.head(5)

,escuela,sexo,edad,entorno,TamFam,EstPadres,Medu,Pedu,Mtrab,Ptrab,...,TiempoLib,SalAm,AlcSem,AlcFin,salud,faltas,asignatura,T1,T2,T3
0,IC,M,19,U,>=4,J,2.0,1.0,casa,otros,...,4,3,1.0,3,5,210.910377,L,8,9,9
1,BG,F,18,U,>=4,J,4.0,4.0,sanidad,sanidad,...,4,4,1.0,1,4,15.000000,M,9,8,8
2,BG,F,16,R,>=4,J,4.0,4.0,sanidad,docencia,...,4,4,2.0,3,4,0.000000,L,17,16,16
3,BG,F,16,U,<4,J,4.0,3.0,docencia,servicios,...,4,3,1.0,2,1,2.000000,L,16,15,16
4,BG,M,18,U,<4,J,3.0,3.0,servicios,sanidad,...,2,4,2.0,4,4,13.000000,M,6,6,8


In [4]:
# Vemos cuáles son las columnas de nuestro dataset
df_train.columns

Index(['escuela', 'sexo', 'edad', 'entorno', 'TamFam', 'EstPadres', 'Medu',
       'Pedu', 'Mtrab', 'Ptrab', 'razon', 'tutor', 'TiempoViaje',
       'TiempoEstudio', 'suspensos', 'apoyo', 'ApFam', 'academia', 'extras',
       'enfermeria', 'EstSup', 'internet', 'pareja', 'RelFam', 'TiempoLib',
       'SalAm', 'AlcSem', 'AlcFin', 'salud', 'faltas', 'asignatura', 'T1',
       'T2', 'T3'],
      dtype='object')

In [5]:
# Extraemos cúales son las variables que tienen valores categóricos y queremos codificar
indices_columnas_categoricas = np.where(df_train.dtypes == object)
columnas_categoricas = list(df_train.columns[indices_columnas_categoricas])
columnas_categoricas

['escuela',
 'sexo',
 'entorno',
 'TamFam',
 'EstPadres',
 'Mtrab',
 'Ptrab',
 'razon',
 'tutor',
 'apoyo',
 'ApFam',
 'academia',
 'extras',
 'enfermeria',
 'EstSup',
 'internet',
 'pareja',
 'asignatura']

In [6]:
# Extraemos las columnas ordinales
columnas_ordinales = [
    col for col in df_train.columns
    if set(df_train[col].dropna().unique()).issubset({0, 1, 2, 3, 4, 5})
]

# Mostramos las columnas ordinales
columnas_ordinales

['Medu',
 'Pedu',
 'TiempoViaje',
 'TiempoEstudio',
 'suspensos',
 'RelFam',
 'TiempoLib',
 'SalAm',
 'AlcSem',
 'AlcFin',
 'salud']

### Exploramos y gestionamos valores nulos o faltantes:

In [7]:
# Exploramos los valores nulos o faltantes del dataframe de train
for col in df_train.columns:
    nulos = df_train[col].isna().sum()
    if nulos > 0:
        print(f"Feature: {col} --> Na Values: {nulos}")


Feature: Medu --> Na Values: 52
Feature: Pedu --> Na Values: 104
Feature: TiempoEstudio --> Na Values: 16
Feature: RelFam --> Na Values: 17
Feature: AlcSem --> Na Values: 20


In [8]:
# Vemos qué tipos de datos hay en las columnas
tipos_dato = set()
for col in df_train.columns:
    tipos_dato.add(df_train[col].dtype)
    
tipos_dato

{dtype('int64'), dtype('float64'), dtype('O')}

In [9]:
# Trabajamos con los valores nulos o faltantes del dataset de train
df_train = gestionar_valores_nulos(X=df_train)

In [10]:
# Comprobamos que no quedan valores nulos o faltantes en el dataframe de train
for col in df_train.columns:
    nulos = df_train[col].isna().sum()
    if nulos > 0:
        print(f"Feature: {col} --> Na Values: {nulos}")

Nos damos cuenta también, de que hay algunos valores en la columna de `razon`, que tienen el valor de "otros" o de "otras". Consideramos que esto es un fallo de transcripción puesto que los valores "otras" aparecen mucho menos que los valores "otros". Por lo tanto los modificamos a mano para dejar los datos consistentes. Imputaremos todos los valores que aparezcan como "otras", dejándolos como "otros".

In [11]:
for col in columnas_categoricas:
    print(f"{col}: {list(df_train[col].unique())} ")


escuela: ['IC', 'BG'] 
sexo: ['M', 'F'] 
entorno: ['U', 'R'] 
TamFam: ['>=4', '<4'] 
EstPadres: ['J', 'S'] 
Mtrab: ['casa', 'sanidad', 'docencia', 'servicios', 'otros'] 
Ptrab: ['otros', 'sanidad', 'docencia', 'servicios', 'casa'] 
razon: ['optativas', 'reputacion', 'otras', 'cercania', 'otros'] 
tutor: ['otros', 'padre', 'madre'] 
apoyo: ['no', 'si'] 
ApFam: ['no', 'si'] 
academia: ['no', 'si'] 
extras: ['si', 'no'] 
enfermeria: ['no', 'si'] 
EstSup: ['no', 'si'] 
internet: ['si', 'no'] 
pareja: ['si', 'no'] 
asignatura: ['L', 'M'] 


In [12]:
# Imputamos el valor de "otros" en aquellos lugares donde pone "otras"
df_train = eliminar_anomalias_razon(df_train)

# Comprobamos que ya no hay la anomalía de "otras"
print(f"{col}: {list(df_train["razon"].unique())} ")

asignatura: ['optativas', 'reputacion', 'otros', 'cercania'] 


Sin embargo en las columnas ordinales, parece no haber ningún valor fuera del rango establecido, ni nada de lo que nos debamos preocupar. Algo que podemos observar, es que en la columna de suspensos, no hay ningún valor de 4, es decir, que nadie ha tenido más de 4 suspensos en el curso anterior.

In [13]:
for col in columnas_ordinales:
    print(f"{col}: {list(df_train[col].unique())} ")

Medu: [2.0, 4.0, 3.0, 1.0, 0.0] 
Pedu: [1.0, 4.0, 3.0, 2.0, 0.0] 
TiempoViaje: [2, 1, 3, 4] 
TiempoEstudio: [1.0, 2.0, 3.0, 4.0] 
suspensos: [3, 1, 0, 2] 
RelFam: [4.0, 2.0, 5.0, 3.0, 1.0] 
TiempoLib: [4, 2, 3, 1, 5] 
SalAm: [3, 4, 2, 5, 1] 
AlcSem: [1.0, 2.0, 3.0, 5.0, 4.0] 
AlcFin: [3, 1, 2, 4, 5] 
salud: [5, 4, 1, 2, 3] 


### Codificamos las variables categóricas del dataset:

Aplicaremos dummy encoding para tener menos columnas que si hiciéramos one-hot encoding, para ello empleamos una función que hemos programado en funciones.py

In [14]:
# Vemos cómo está el dataframe antes de ser codificado con dummy encoding
df_train.head(5)

,escuela,sexo,edad,entorno,TamFam,EstPadres,Medu,Pedu,Mtrab,Ptrab,...,TiempoLib,SalAm,AlcSem,AlcFin,salud,faltas,asignatura,T1,T2,T3
0,IC,M,19,U,>=4,J,2.0,1.0,casa,otros,...,4,3,1.0,3,5,210.910377,L,8,9,9
1,BG,F,18,U,>=4,J,4.0,4.0,sanidad,sanidad,...,4,4,1.0,1,4,15.000000,M,9,8,8
2,BG,F,16,R,>=4,J,4.0,4.0,sanidad,docencia,...,4,4,2.0,3,4,0.000000,L,17,16,16
3,BG,F,16,U,<4,J,4.0,3.0,docencia,servicios,...,4,3,1.0,2,1,2.000000,L,16,15,16
4,BG,M,18,U,<4,J,3.0,3.0,servicios,sanidad,...,2,4,2.0,4,4,13.000000,M,6,6,8


In [15]:
# Codificamos las variables categóricas del dataset
df_train_encoded = df_train.copy() 
df_train_encoded = dummy_encoding(X=df_train_encoded, categorical_columns=columnas_categoricas)
df_train_encoded.head()

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
0,19,2.0,1.0,2,1.0,3,4.0,4,3,1.0,...,0,0,0,0,1,0,0,1,1,0
1,18,4.0,4.0,1,2.0,1,2.0,4,4,1.0,...,1,1,1,0,1,1,1,1,1,1
2,16,4.0,4.0,1,2.0,0,2.0,4,4,2.0,...,0,0,1,0,1,1,1,0,0,0
3,16,4.0,3.0,3,2.0,0,5.0,4,3,1.0,...,0,0,1,0,1,1,1,1,0,0
4,18,3.0,3.0,1,2.0,1,3.0,2,4,2.0,...,1,0,1,1,0,1,1,1,0,1


In [16]:
# Comrpobamos como nos quedan las columnas tras codificar con dummy
df_train_encoded.columns

Index(['edad', 'Medu', 'Pedu', 'TiempoViaje', 'TiempoEstudio', 'suspensos',
       'RelFam', 'TiempoLib', 'SalAm', 'AlcSem', 'AlcFin', 'salud', 'faltas',
       'T1', 'T2', 'T3', 'escuela_IC', 'sexo_M', 'entorno_U', 'TamFam_>=4',
       'EstPadres_S', 'Mtrab_docencia', 'Mtrab_otros', 'Mtrab_sanidad',
       'Mtrab_servicios', 'Ptrab_docencia', 'Ptrab_otros', 'Ptrab_sanidad',
       'Ptrab_servicios', 'razon_optativas', 'razon_otros', 'razon_reputacion',
       'tutor_otros', 'tutor_padre', 'apoyo_si', 'ApFam_si', 'academia_si',
       'extras_si', 'enfermeria_si', 'EstSup_si', 'internet_si', 'pareja_si',
       'asignatura_M'],
      dtype='object')

### Valores físicamente imposibles:

Nos damos cuenta de que por ejemplo en la columna de **faltas**, aparecen valores decimales, lo cual es imposible ya que las ausencias a clase tienen que ser números enteros. Por lo tanto, asumimos que ha podido ser un error de transcripción de los datos o cualquier otro tipo de error en el procesado de los datos, y los convertimos a un formato coherente.

In [17]:
# Llamamos a la función que convierte a enteros todos los valores de la columna de faltas
df_train_encoded = ajustar_valores_decimales_faltas(df_train_encoded)

También comprobamos si existe algún valor negativo en alguna columna, lo cual no tendría ningún sentido.

In [18]:
for col in df_train_encoded.columns:
    for elem in df_train_encoded[col]:
        if elem < 0:
            print(col)
            print(elem)
            print()

### Exploramos y gestionamos los valores atípicos / outliers:

En el fichero de exploración de datos, hemos observado que la variable de faltas tiene outliers con valores notablemente altos, por lo que vamos a ajustar dichos valores de forma que podamos trabajar con ellos de una forma más robusta, sin tampoco elimiar la información que nos aportan ya que puede ser muy valiosa para que mejore el rendimiento del modelo. Para ello empleamos una función que hemos programado en el fichero de funciones.py

In [19]:
# Vemos cuales son los valores máximos de faltas (valores físicamente imposibles teniendo en cuenta los días lectivos que hay en un año)
df_train_encoded.query("faltas > 100")["faltas"]

0      210
32     243
79     233
98     246
104    202
178    204
215    212
227    237
259    213
296    243
328    211
343    222
350    239
372    239
530    224
554    207
586    200
654    208
694    231
790    217
Name: faltas, dtype: int64

In [20]:
# Aplicamos el "upper clip"
th_superior = 150
df_train_encoded = ajustar_outliers_faltas(X=df_train_encoded, th=th_superior)

In [21]:
# Comprobamos que todo ha ido bien
df_train_encoded.query("faltas > 100")["faltas"]

0      150
32     150
79     150
98     150
104    150
178    150
215    150
227    150
259    150
296    150
328    150
343    150
350    150
372    150
530    150
554    150
586    150
654    150
694    150
790    150
Name: faltas, dtype: int64

### Dividimos el dataset en conjunto de train y de validation:

Dividir el conjunto de datos en train y validation es muy importante, debemos hacerlo antes de empezar a entrenar. Esto nos permite entrenar el modelo en un subconjunto de los datos (train) y evaluarlo en un subconjunto separado (validation). De esta manera, podemos medir el rendimiento del modelo en datos no vistos, lo que ayuda a prevenir el sobreajuste y garantiza que el modelo generalice bien a nuevos datos.

In [22]:
df_train, df_val = dividir_train_validation(df=df_train_encoded, random_state=10, proporcion_train=0.70)

In [23]:
df_train

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
0,18,2.0,2.0,2,2.0,1,5.0,5,5,1.0,...,0,0,0,0,0,1,0,1,1,0
1,18,4.0,4.0,1,2.0,0,5.0,4,3,1.0,...,0,0,1,1,1,1,1,1,0,1
2,17,2.0,2.0,1,2.0,0,4.0,2,2,1.0,...,0,0,1,0,0,1,1,0,1,1
3,16,3.0,3.0,2,1.0,0,5.0,4,2,1.0,...,1,0,0,0,1,1,1,1,0,1
4,15,3.0,4.0,1,2.0,0,5.0,5,1,1.0,...,0,0,1,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,17,2.0,2.0,1,2.0,1,3.0,3,1,1.0,...,1,0,1,0,0,1,1,1,1,1
580,17,1.0,2.0,1,1.0,0,5.0,5,1,1.0,...,1,0,0,0,0,1,1,1,0,0
581,16,2.0,2.0,3,2.0,0,5.0,3,4,1.0,...,1,0,1,0,1,1,1,1,0,0
582,18,1.0,1.0,2,2.0,0,2.0,3,5,1.0,...,0,1,0,0,0,1,1,0,0,0


In [24]:
df_val

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
584,19,3.0,1.0,2,2.0,3,3.0,5,4,1.0,...,0,0,1,0,1,1,0,0,1,0
585,18,1.0,2.0,2,1.0,0,3.0,4,4,2.0,...,0,0,0,0,0,0,0,1,0,0
586,15,4.0,4.0,1,2.0,0,4.0,4,4,1.0,...,1,1,1,0,1,0,1,1,0,0
587,15,4.0,2.0,1,2.0,0,4.0,3,3,1.0,...,0,0,1,1,0,1,1,1,0,1
588,15,2.0,2.0,1,1.0,0,4.0,3,1,1.0,...,0,1,1,0,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,15,4.0,3.0,1,2.0,0,5.0,2,2,1.0,...,0,1,1,1,1,1,1,1,1,0
831,16,2.0,2.0,1,2.0,0,4.0,3,5,2.0,...,1,0,0,0,0,1,0,1,0,0
832,16,4.0,4.0,1,2.0,0,4.0,2,4,2.0,...,1,0,1,0,1,1,1,1,0,0
833,17,1.0,3.0,3,2.0,1,5.0,2,4,1.0,...,1,0,1,0,1,1,1,1,0,0


Vemos que el número de filas de train sumado al número de filas de validación, da en total el número de filas que tenía inicialmente el dataframe, por lo que parece que hemos hecho bien la división de los datasets, además se ve que validación tiene menor cantidad de filas, lo cual se debe a la proporción de split que hemos elegido, pero que se podría modificar.

### Estandarizamos las variables de cada conjunto de datos:

Antes de entrenar los modelos, es fundamental estandarizar las variables de entrada para garantizar que todas estén en la misma escala. Esto es especialmente importante cuando se utilizan algoritmos que son sensibles a la magnitud de los valores, como la regresión lineal, KNN, SVM o redes neuronales. Estandarizar consiste en transformar cada variable para que tenga media cero y desviación típica uno.

La estandarización debe hacerse únicamente con los parámetros del conjunto de entrenamiento (media y desviación típica). Posteriormente, se aplica esa misma transformación al conjunto de validación o test. De esta forma evitamos que el modelo tenga acceso indirecto a información del conjunto de train, lo cual iría en contra de la independencia entre los conjuntos y podría aumentar artificialmente el rendimiento del modelo. Planteando así la estandarización, evitamos que variables con escalas naturalmente más grandes dominen el proceso de optimización, lo cual podría distorsionar los resultados y el aprendizaje del modelo.

Cabe destacar también, que la columna de target, es decir, **`T3`, no la estandarizamos.**

In [25]:
# Elegimos qué columnas queremos estandarizar (solo las numéricas que no sean de 0 y 1, es decir incluyendo ordinales, pero excluyendo T3)
columnas_estandarizar = [
    col for col in df_train_encoded.columns
    if col != variable_objetivo and not set(df_train_encoded[col].dropna().unique()).issubset({0, 1})
]

columnas_estandarizar

['edad',
 'Medu',
 'Pedu',
 'TiempoViaje',
 'TiempoEstudio',
 'suspensos',
 'RelFam',
 'TiempoLib',
 'SalAm',
 'AlcSem',
 'AlcFin',
 'salud',
 'faltas',
 'T1',
 'T2']

In [26]:
# Extraemos en un diccionario, la media y la std de cada columna del conjunto de train, lo cual vamos a usar para normalizar
stats_estandarizacion = sacar_escalado_train(df_train, columnas_estandarizar) # devuelve un dict

#### Estandarización del conjunto de train:

In [27]:
# Vemos cúales son las estadísticas antes de estandarizar
df_train.describe()

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
count,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,...,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,16.741438,2.684932,2.359589,1.532534,1.962329,0.268836,3.921233,3.152397,3.095890,1.455479,...,0.217466,0.113014,0.621575,0.227740,0.501712,0.811644,0.916096,0.779110,0.352740,0.385274
std,1.190267,1.089500,1.041577,0.714205,0.837757,0.647985,0.926434,1.050555,1.151202,0.877430,...,0.412876,0.316881,0.485410,0.419733,0.500426,0.391332,0.277482,0.415202,0.478232,0.487077
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,20.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
# Restamos la media y dividimos entre la desviación típica, para cada dato y cada columna (usando las stats de train)
for col in columnas_estandarizar:
    df_train[col] = (df_train[col] - stats_estandarizacion[col]["mean"]) / stats_estandarizacion[col]["std"]

# Volvemos a comprobar cuáles son las estadísticas para ver que ha ido bien el proceso de estandarización
df_train.describe()

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
count,5.840000e+02,5.840000e+02,584.000000,5.840000e+02,5.840000e+02,5.840000e+02,5.840000e+02,5.840000e+02,5.840000e+02,5.840000e+02,...,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,-1.840233e-16,-8.516779e-17,0.000000,4.258390e-17,6.539670e-17,1.292725e-17,-6.083414e-18,-1.125432e-16,6.539670e-17,7.300097e-17,...,0.217466,0.113014,0.621575,0.227740,0.501712,0.811644,0.916096,0.779110,0.352740,0.385274
std,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.412876,0.316881,0.485410,0.419733,0.500426,0.391332,0.277482,0.415202,0.478232,0.487077
min,-1.463065e+00,-2.464370e+00,-2.265401,-7.456321e-01,-1.148696e+00,-4.148793e-01,-3.153203e+00,-2.048820e+00,-1.820610e+00,-5.191061e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.229175e-01,-6.286657e-01,-0.345235,-7.456321e-01,-1.148696e+00,-4.148793e-01,8.502188e-02,-1.450636e-01,-9.519532e-01,-5.191061e-01,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,2.172299e-01,2.891862e-01,-0.345235,-7.456321e-01,4.496676e-02,-4.148793e-01,8.502188e-02,-1.450636e-01,-8.329590e-02,-5.191061e-01,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,1.057377e+00,1.207038e+00,0.614848,6.545259e-01,4.496676e-02,-4.148793e-01,1.164430e+00,8.068146e-01,7.853614e-01,6.205854e-01,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,2.737672e+00,1.207038e+00,1.574930,3.454842e+00,2.432293e+00,4.214856e+00,1.164430e+00,1.758693e+00,1.654019e+00,4.039660e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Estandarización del conjunto de validation:

In [29]:
# Vemos cúales son las estadísticas antes de estandarizar
df_val.describe()

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,...,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,16.788845,2.573705,2.358566,1.494024,1.948207,0.318725,3.872510,3.314741,3.219124,1.581673,...,0.243028,0.123506,0.617530,0.155378,0.501992,0.792829,0.900398,0.828685,0.374502,0.362550
std,1.365004,1.068431,1.076530,0.706374,0.785689,0.749672,0.975542,0.975982,1.136571,0.986054,...,0.429769,0.329674,0.486961,0.362989,0.500995,0.406089,0.300066,0.377537,0.484961,0.481697
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,3.500000,3.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,18.000000,3.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
# Restamos la media y dividimos entre la desviación típica, para cada dato y cada columna (usando las stats de train)
for col in columnas_estandarizar:
    df_val[col] = (df_val[col] - stats_estandarizacion[col]["mean"]) / stats_estandarizacion[col]["std"]
 
# Volvemos a comprobar cuáles son las estadísticas para ver que ha ido bien el proceso de estandarizaciín
df_val.describe()

,edad,Medu,Pedu,TiempoViaje,TiempoEstudio,suspensos,RelFam,TiempoLib,SalAm,AlcSem,...,tutor_padre,apoyo_si,ApFam_si,academia_si,extras_si,enfermeria_si,EstSup_si,internet_si,pareja_si,asignatura_M
count,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,...,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000,251.000000
mean,0.039828,-0.102089,-0.000982,-0.053921,-0.016856,0.076992,-0.052592,0.154532,0.107047,0.143822,...,0.243028,0.123506,0.617530,0.155378,0.501992,0.792829,0.900398,0.828685,0.374502,0.362550
std,1.146804,0.980662,1.033558,0.989035,0.937848,1.156928,1.053008,0.929016,0.987290,1.123798,...,0.429769,0.329674,0.486961,0.362989,0.500995,0.406089,0.300066,0.377537,0.484961,0.481697
min,-1.463065,-2.464370,-2.265401,-0.745632,-1.148696,-0.414879,-3.153203,-2.048820,-1.820610,-0.519106,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-0.622918,-0.628666,-0.345235,-0.745632,-1.148696,-0.414879,-0.454682,-0.145064,-0.951953,-0.519106,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000
50%,0.217230,0.289186,-0.345235,-0.745632,0.044967,-0.414879,0.085022,-0.145064,-0.083296,-0.519106,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,1.057377,0.289186,0.614848,0.654526,0.044967,-0.414879,1.164430,0.806815,0.785361,0.620585,...,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,4.417967,1.207038,1.574930,3.454842,2.432293,4.214856,1.164430,1.758693,1.654019,4.039660,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Guardamos los datos ya procesados en ficheros CSV:

In [31]:
# Elegimos los nombres de las rutas donde vamos a guardar los ficheros
nombre_carpeta = "processed_data"
nombre_train = "df_train"
nombre_val = "df_val"

# Guardamos los ficheros en csv
guardar_df_to_csv(X=df_train, carpeta=nombre_carpeta, nombre=nombre_train)
guardar_df_to_csv(X=df_val, carpeta=nombre_carpeta, nombre=nombre_val)